In [1]:
# !pip install keras_vggface
# !pip install keras_applications
# !pip install deepface

In [17]:
from collections import defaultdict
from glob import glob
from random import choice, sample

import cv2
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.layers import Input, Dense, GlobalMaxPool2D, GlobalAvgPool2D, Concatenate, Multiply, Dropout, Subtract, LayerNormalization, BatchNormalization
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import L2

from deepface.commons.functions import find_input_shape, normalize_input
from deepface.DeepFace import build_model

from tqdm import tqdm

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [24]:
# Hyperparameters
BASE_MODEL = 'DeepFace'
IGNORE_TOP_NLAYERS_ARCH = 2 #-11 #6
IGNORE_BOTTOM_NLAYERS_TUNE = -2 #15
IGNORE_TOP_NLAYERS_TUNE = 0
NORMALIZATION = 'base'
FINE_TUNE = False

# Modify paths as per your method of saving them
BASE_PATH = "/root/KinshipRecognition"
TRAIN_FILE_PATH = f"{BASE_PATH}/data/aug_train_ds.csv"
TRAIN_FOLDERS_PATH = f"{BASE_PATH}/data/train/train-faces/"

# All images belonging to families F09** will be used to create the validation set while training the model
# For final submission, you can add these to the training data as well
val_families_list = ["F06"]
# val_families_list = ["F09","F04","F08","F06", "F02"]

# Get input shape and normalization method.
input_shape = find_input_shape(build_model(BASE_MODEL))
if NORMALIZATION not in ['raw', 'Facenet512', 'DeepFace', 'VGGFace', 'VGGFace2', 'ArcFace']:
    NORMALIZATION = 'base' 


In [25]:
def get_train_val(family_name):

    val_families = family_name

    all_images = glob(TRAIN_FOLDERS_PATH + "*/*/*.jpg")
    train_images = [x for x in all_images if val_families not in x]
    val_images = [x for x in all_images if val_families in x]

    train_person_to_images_map = defaultdict(list)

    ppl = [x.split("/")[-3] + "/" + x.split("/")[-2] for x in all_images]

    for x in train_images:
        train_person_to_images_map[x.split("/")[-3] + "/" + x.split("/")[-2]].append(x)

    val_person_to_images_map = defaultdict(list)

    for x in val_images:
        val_person_to_images_map[x.split("/")[-3] + "/" + x.split("/")[-2]].append(x)
    relationships = pd.read_csv(TRAIN_FILE_PATH)
    relationships = list(zip(relationships.p1.values, relationships.p2.values, relationships.relationship.values))
    relationships = [(x[0],x[1],x[2]) for x in relationships if x[0][:10] in ppl and x[1][:10] in ppl]    

    train = [x for x in relationships if val_families not in x[0]]
    val = [x for x in relationships if val_families in x[0]]
    return train, val, train_person_to_images_map, val_person_to_images_map

In [26]:
def read_img(path, input_shape, normalization='base'):
    img = cv2.imread(path, -1)
    img = cv2.resize(img, input_shape)
    img = cv2.normalize(img,  np.zeros(img.shape[:2]), 0, 255, cv2.NORM_MINMAX)
    img = normalize_input(img, normalization=normalization)
    return np.array(img).astype(np.float)

In [27]:
def gen(list_tuples, person_to_images_map, input_shape, batch_size=16, normalization='base'):
    ppl = list(person_to_images_map.keys())
    while True:
        batch_tuples = sample(list_tuples, batch_size)
        
        # All the samples are taken from train_ds.csv, labels are in the labels column
        labels = []
        for tup in batch_tuples:
            labels.append(tup[2])
        labels = np.array(labels)

        # Original images preprocessed
        X1 = [x[0] for x in batch_tuples]
        X1 = np.array([read_img(train_folders_path + x, input_shape, normalization=NORMALIZATION) for x in X1])
        
        X2 = [x[1] for x in batch_tuples]
        X2 = np.array([read_img(train_folders_path + x, input_shape, normalization=NORMALIZATION) for x in X2])
        
        # Mirrored images
        X1_mirror = np.asarray([cv2.flip(x, 1) for x in X1])
        X2_mirror = np.asarray([cv2.flip(x, 1) for x in X2])
        X1 = np.r_[X1, X1_mirror]
        X2 = np.r_[X2, X2_mirror]
        
        yield [X1, X2], np.r_[labels, labels]

In [28]:
def baseline_model(model_name, fine_tune=True):
    input_1 = Input(shape=input_shape + (3,))
    input_2 = Input(shape=input_shape + (3,))

    backbone = Sequential()
    for layer in build_model(model_name).layers[:-IGNORE_TOP_NLAYERS_ARCH]:
        backbone.add(layer)
    for x in backbone.layers:
        x.trainable = False

    if fine_tune:
        for x in backbone.layers[:IGNORE_BOTTOM_NLAYERS_TUNE]:
            x.trainable = False
        if IGNORE_TOP_NLAYERS_TUNE == 0:
            for x in backbone.layers[IGNORE_BOTTOM_NLAYERS_TUNE:]:
                x.trainable = True
        else:
            for x in backbone.layers[IGNORE_BOTTOM_NLAYERS_TUNE:-IGNORE_TOP_NLAYERS_TUNE]:
                x.trainable = True

    x1 = backbone(input_1)
    x2 = backbone(input_2)

    x1 = GlobalAvgPool2D()(x1)
    x2 = GlobalAvgPool2D()(x2)

    x1 = LayerNormalization(axis=-1, epsilon=0.001, center=False, scale=False)(x1)
    x2 = LayerNormalization(axis=-1, epsilon=0.001, center=False, scale=False)(x2)

    x3 = Subtract()([x1, x2])
    x3 = Multiply()([x3, x3])
    x1_ = Multiply()([x1, x1])
    x2_ = Multiply()([x2, x2])
    x4 = Subtract()([x1_, x2_])
    x5 = Multiply()([x1, x2])
    x = Concatenate(axis=-1)([x3, x4, x5])
        
#     x = LayerNormalization(axis=-1, epsilon=0.001, center=True, scale=True)(x)
    x = Dense(32, activation="relu")(x)
    x = Dropout(0.05)(x)    
    x = Dense(128, activation="relu")(x)
    x = Dropout(0.05)(x)    
    x = Dense(32, activation="tanh")(x)
#     x = LayerNormalization(axis=-1, epsilon=0.001, center=True, scale=False)(x)
    x = Dropout(0.05)(x)    
    out = Dense(1, kernel_regularizer=L2(.01), activation="sigmoid")(x)

    model = Model([input_1, input_2], out)
    model.compile(loss="binary_crossentropy", metrics=['acc'], optimizer=Adam(0.00001))
    model.summary()

    return model


In [29]:
for i in tqdm(range(len(val_families_list))):
    train, val, train_person_to_images_map, val_person_to_images_map = get_train_val(val_families_list[i])
    file_path = f"{BASE_PATH}/log/model/vgg_face_{i}.h5"
    checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    reduce_on_plateau = ReduceLROnPlateau(monitor="val_acc", mode="max", factor=0.3, patience=30, verbose=1)
    callbacks_list = [checkpoint, reduce_on_plateau]
    
    model = baseline_model(BASE_MODEL, fine_tune=FINE_TUNE)
    
    history = model.fit(gen(train, train_person_to_images_map, input_shape, batch_size=16, normalization=NORMALIZATION), 
                        validation_data=gen(val, val_person_to_images_map, input_shape, batch_size=16, normalization=NORMALIZATION), 
                        epochs=25, steps_per_epoch=300, validation_steps=200,
                        verbose=1, callbacks=callbacks_list, 
                        use_multiprocessing=False, workers=1)

  0%|                                                                              | 0/1 [00:42<?, ?it/s]

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 152, 152, 3) 0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            [(None, 152, 152, 3) 0                                            
__________________________________________________________________________________________________
sequential_10 (Sequential)      (None, 21, 21, 16)   73507392    input_7[0][0]                    
                                                                 input_8[0][0]                    
__________________________________________________________________________________________________
global_average_pooling2d (Globa (None, 16)           0           sequential_10[0][0]        

TypeError: gen() missing 1 required positional argument: 'input_shape'

In [10]:
test_path = f"{BASE_PATH}/data/test/"
submission = pd.read_csv(f'{BASE_PATH}/data/test_ds.csv')

preds_for_sub = np.zeros(submission.shape[0])
all_preds = list()
for i in tqdm(range(len(val_families_list))):
    file_path = f"{BASE_PATH}/vgg_face_{i}.h5"
    model.load_weights(file_path)
    # Get the predictions
    predictions = []

    for j in range(0, len(submission.p1.values), 32):
        X1 = submission.p1.values[j:j+32]
        X1 = np.array([read_img(test_path + x) for x in X1])

        X2 = submission.p2.values[j:j+32]
        X2 = np.array([read_img(test_path + x) for x in X2])

        pred = model.predict([X1, X2]).ravel().tolist()
        # pred = model([X1, X2], training=True).numpy().ravel().tolist()
        predictions += pred    
    
    all_preds.append(np.array(predictions))
    preds_for_sub += np.array(predictions) / len(val_families_list)

all_preds = np.asarray(all_preds)
submission['score'] = preds_for_sub
submission.to_csv(f"{BASE_PATH}/log/results/aug_vggface_notune_dense32-128-32_F06.csv", index=False)

  0%|          | 0/1 [00:00<?, ?it/s]

In [12]:
print(np.sum(pred_for_sub <= 0.5))
print(len(pred_for_sub) + '\n')
for line in preds_for_sub:
    print(line)

0.12341336905956268
0.06516876816749573
0.6288173794746399
0.5620527267456055
0.17016741633415222
0.7743032574653625
0.2531704306602478
0.033472392708063126
0.8822365999221802
0.44409772753715515
0.048788852989673615
0.24944539368152618
0.48926278948783875
0.31238213181495667
0.015374405309557915
0.8201798796653748
0.0717521458864212
0.39531970024108887
0.2700093984603882
0.08493220061063766
0.8402065634727478
0.04393022507429123
0.35728922486305237
0.16254207491874695
0.2342737913131714
0.17211395502090454
0.1630638837814331
0.6988136768341064
0.14660023152828217
0.3256670832633972
0.06583888083696365
0.535265326499939
0.6969621181488037
0.19694769382476807
0.2023462951183319
0.08944737911224365
0.7729066610336304
0.11089440435171127
0.8335807919502258
0.5303354859352112
0.5169271230697632
0.0453726090490818
0.7651994228363037
0.7724387645721436
0.792084813117981
0.14664749801158905
0.09810520708560944
0.07899443060159683
0.05096563696861267
0.12377625703811646
0.08123815059661865
0.0